In [18]:
from glob import glob
import os
import numpy as np
import cv2
from time import time
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping

In [19]:

# Specify the path to your dataset
dataset_path = "./training"

# Read images and labels from the dataset
images = []
labels = []

# Define the desired dimensions for your images
desired_width = 100
desired_height = 100
print(images)
print(labels)

[]
[]


In [20]:
for person_id, person_folder in enumerate(sorted(glob(os.path.join(dataset_path, "*")))):
    person_name = os.path.basename(person_folder)
    for image_path in glob(os.path.join(person_folder, "*.jpg")):
        # Read the image using OpenCV
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        # Resize the image to the desired dimensions
        image = cv2.resize(image, (desired_width, desired_height))
        # Flatten the image and add it to the list
        images.append(image.flatten())
        # Extract the person's name from the filename and add it to the labels
        person_name_in_filename = os.path.splitext(os.path.basename(image_path))[0].split("_")[0]
        labels.append(person_name_in_filename)

In [21]:

# Convert the lists to numpy arrays
X = np.array(images)
y = np.array(labels)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [22]:
# Reshape the flattened images to 2D arrays
X_train_reshaped = X_train.reshape(-1, desired_height, desired_width, 1)
X_test_reshaped = X_test.reshape(-1, desired_height, desired_width, 1)

# Convert labels to numerical values
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Normalize pixel values to be between 0 and 1
X_train_normalized = X_train_reshaped / 255.0
X_test_normalized = X_test_reshaped / 255.0

X_train_cnn = X_train_normalized
X_val_cnn = X_test_normalized
y_train_cnn = y_train_encoded
y_val_cnn = y_test_encoded

In [23]:
# Define the CNN model
model = keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(desired_height, desired_width, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(len(np.unique(y)), activation='softmax')
])


In [24]:
# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Early stopping callback to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)


In [11]:
# Train the CNN model
history = model.fit(X_train_cnn, y_train_cnn, epochs=20, validation_data=(X_val_cnn, y_val_cnn), callbacks=[early_stopping])

Epoch 1/20


2024-02-11 21:49:32.853903: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 30720000 exceeds 10% of free system memory.
2024-02-11 21:49:33.793482: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 26214400 exceeds 10% of free system memory.


24/24 [==============================] - 14s 450ms/step - loss: 1.6042 - accuracy: 0.3854 - val_loss: 1.1034 - val_accuracy: 0.5781
Epoch 2/20
24/24 [==============================] - 12s 489ms/step - loss: 0.9032 - accuracy: 0.6706 - val_loss: 0.7188 - val_accuracy: 0.7656
Epoch 3/20
24/24 [==============================] - 11s 441ms/step - loss: 0.5708 - accuracy: 0.8112 - val_loss: 0.5547 - val_accuracy: 0.8021
Epoch 4/20
24/24 [==============================] - 13s 522ms/step - loss: 0.3986 - accuracy: 0.8620 - val_loss: 0.3427 - val_accuracy: 0.9115
Epoch 5/20
24/24 [==============================] - 12s 495ms/step - loss: 0.2381 - accuracy: 0.9310 - val_loss: 0.2801 - val_accuracy: 0.9062
Epoch 6/20
24/24 [==============================] - 12s 487ms/step - loss: 0.1467 - accuracy: 0.9557 - val_loss: 0.2311 - val_accuracy: 0.9323
Epoch 7/20
24/24 [==============================] - 12s 487ms/step - loss: 0.1114 - accuracy: 0.9635 - val_loss: 0.2239 - val_accuracy: 0.9583
Epoch 8/20

In [25]:
t0 = time()
training = model.fit(X_train_cnn, y_train_cnn, epochs=20, validation_data=(X_val_cnn, y_val_cnn), callbacks=[early_stopping])
print("done in %0.3fs" % (time() - t0))

Epoch 1/20
12/12 [==============================] - 8s 579ms/step - loss: 1.7729 - accuracy: 0.2005 - val_loss: 1.6388 - val_accuracy: 0.4479
Epoch 2/20
12/12 [==============================] - 6s 481ms/step - loss: 1.4143 - accuracy: 0.5156 - val_loss: 1.2344 - val_accuracy: 0.4896
Epoch 3/20
12/12 [==============================] - 5s 449ms/step - loss: 1.0305 - accuracy: 0.6068 - val_loss: 0.8402 - val_accuracy: 0.7083
Epoch 4/20
12/12 [==============================] - 6s 505ms/step - loss: 0.7974 - accuracy: 0.7188 - val_loss: 0.7222 - val_accuracy: 0.7604
Epoch 5/20
12/12 [==============================] - 6s 488ms/step - loss: 0.5596 - accuracy: 0.8099 - val_loss: 0.8829 - val_accuracy: 0.7500
Epoch 6/20
12/12 [==============================] - 7s 544ms/step - loss: 0.4257 - accuracy: 0.8385 - val_loss: 0.7003 - val_accuracy: 0.7604
Epoch 7/20
12/12 [==============================] - 11s 914ms/step - loss: 0.3262 - accuracy: 0.8932 - val_loss: 0.7259 - val_accuracy: 0.7396
Epoch

In [13]:
t0 = time()
test_loss, test_accuracy = model.evaluate(X_test_normalized, y_test_encoded)
t1 = time()

print(f'Test Accuracy (CNN): {test_accuracy:.2f}')
print(f"Time taken: {t1-t0} seconds")

6/6 [==============================] - 1s 145ms/step - loss: 0.1561 - accuracy: 0.9531
Test Accuracy (CNN): 0.95
Time taken: 1.4085814952850342 seconds


In [14]:
t0 = time()
y_pred_cnn_probs = model.predict(X_test_normalized)
print("done in %0.3fs" % (time() - t0))

6/6 [==============================] - 1s 127ms/step
done in 1.543s


In [15]:
# Convert probabilities to class predictions
y_pred_cnn = np.argmax(y_pred_cnn_probs, axis=1)

# Decode numerical predictions back to original labels
y_pred_cnn_labels = label_encoder.inverse_transform(y_pred_cnn)

In [16]:
# Print classification report for CNN
print("Classification Report (CNN):")
print(classification_report(y_test, y_pred_cnn_labels, target_names=np.unique(y)))

# Print confusion matrix for CNN
conf_matrix_cnn = confusion_matrix(y_test, y_pred_cnn_labels, labels=np.unique(y))
print("Confusion Matrix (CNN):")
print(conf_matrix_cnn)

# Calculate and print accuracy, precision, recall, and F1-score for CNN
accuracy_cnn = accuracy_score(y_test, y_pred_cnn_labels)
precision_cnn = precision_score(y_test, y_pred_cnn_labels, average='weighted')
recall_cnn = recall_score(y_test, y_pred_cnn_labels, average='weighted')
f1_cnn = f1_score(y_test, y_pred_cnn_labels, average='weighted')

print(f"Accuracy (CNN): {accuracy_cnn:.2f}")
print(f"Precision (CNN): {precision_cnn:.2f}")
print(f"Recall (CNN): {recall_cnn:.2f}")
print(f"F1 Score (CNN): {f1_cnn:.2f}")

Classification Report (CNN):
              precision    recall  f1-score   support

   adityaroy       1.00      1.00      1.00        37
        bald       0.97      0.88      0.92        33
   devillers       1.00      0.97      0.98        31
        modi       0.93      0.87      0.90        30
        rock       0.82      1.00      0.90        28
     ronaldo       1.00      1.00      1.00        33

    accuracy                           0.95       192
   macro avg       0.95      0.95      0.95       192
weighted avg       0.96      0.95      0.95       192

Confusion Matrix (CNN):
[[37  0  0  0  0  0]
 [ 0 29  0  2  2  0]
 [ 0  1 30  0  0  0]
 [ 0  0  0 26  4  0]
 [ 0  0  0  0 28  0]
 [ 0  0  0  0  0 33]]
Accuracy (CNN): 0.95
Precision (CNN): 0.96
Recall (CNN): 0.95
F1 Score (CNN): 0.95
